# KFold
- 데이터셋을 K개의 동일 크기 부분(Fold)나눠서 K번 반복하여 하나의 폴드를 검증용으로 사용을 하고 나머지 k-1개의 폴드를 학습용으로 사용
- 모든 폴드를 한번은 검증용으로 포함이 되고 k-1개를 학습용으로 사용 
- 데이터가 수가 적을때 모델의 성능을 안정적으로 평가 할 수 있는 방법 

- 매개변수 
    - n_splits
        - 기본값 : 5
        - 폴드의 개수를 지정 
        - 최소 값은 2
    - shuffle
        - 기본값 : False
        - 데이터를 분할하기 전에 섞을지 지정 
        - True로 변경하게 되면 폴드가 랜덤하게 구성 
    - random_state
        - 기본값 : None
        - shuffle이 True인 경우에 사용
        - 랜덤 시드 고정 
- 속성
    - n_splits 
        - 분할된 폴드의 개수
- 메서드 
    - split(x, y = None)
        - 학습용/검증용 인덱스를 생성
        - 반복문을 이용하여 (train_index, test_index)로 변환하여 사용

- 장점 
    - 데이터를 폴드화 해서 학습/ 검증용으로 사용하기 때문에 데이터가 낭비가 없다. 
    - 일반적으로 사용하던 train_test_split보다 성능 평가가 안정적(설명이 충분하다.)
- 단점 
    - K번의 학습 -> K번의 예측 -> K번의 평가 --> 계산이 늘어남 -> 시간 증가
    - 데이터의 크기가 크다면 시간이 증가 

- 변형 KFold 클래스 
    - StratifiedKFold : 분류 문제에서 클래스의 비율을 유지하여 분할 
    - GroupKFold : 그룹 단위로 데이터를 나눠 그룹이 학습/검증에 동시에 들어가지 않도록 보장 
    - RepeatedKFold : KFold를 여러번 반복해 평가 안정성 강화 

In [ ]:
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split, GridSearchCV, \
                KFold, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC, SVR
from sklearn.metrics import classification_report, r2_score

In [ ]:
# Pipeline + GridSearchCV + KFold를 사용하여 분류 문제 해결 
# iris 데이터를 로드 
iris = pd.read_csv("../data/iris.csv")
iris.head()

In [ ]:
# target 데이터를 0, 1, 2로 데이터를 변경
# iris['target'].unique()
for i, key in enumerate(iris['target'].unique()):
    # print(i)
    # print(key)
    iris['target'] = iris['target'].replace(key, i)

In [ ]:
# train, test 데이터셋을 구분 
x = iris.drop('target', axis=1).values
y = iris['target'].values


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(
    x, y, random_state=42, test_size=0.2, stratify=y
)

In [ ]:
# KFold를 이용해서 데이터 분할 
# 분류 모델 -> target의 비율 1:1:1 -> 비율을 맞춰서 폴드화
# stratifiedKFold를 이용
cv_folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
cv_folds

In [ ]:
# Pipeline 생성 -> 
# 학습(fit()) -> Scaler 생성하고 fit()을 이용하여 범위를 지정하고 
# transform()을 이용해서 스케일링을 하고 모델에 학습을 시킨다.
# 예측(predict()) -> 생성된 Scaler를 사용하여 
# 검증 데이터를 transform()을 이용해서 스케일링하고 학습된 모델에 
# predict() 함수를 이용하여 예측값을 반환
pipe = Pipeline(
    [
        ('scaler', StandardScaler()), 
        ('svc', SVC(probability=True, random_state=42))
    ]
)

In [ ]:
# GridSearchCV에서 사용할 파라미터 조합을 생성 
# pipeline을 이용하여 Grid를 사용하면 매개변수의 이름을 수정할 필요가 있다
# pipeline에서 사용할 모델에( 이름 + __ + 매개변수명) 키 값들을 변경 
params = {
    "svc__C" : [0.1, 1, 10], 
    'svc__gamma' : ['scale', 'auto'], 
    'svc__kernel' : ['linear', 'rbf']
}

In [ ]:
grid_cls = GridSearchCV(
    estimator= pipe,   # gird에서 사용할 모델은 pipeline으로 만들어진 모델
    param_grid= params, # dict 형태로 각 파라미터별 사용할 값 지정
    scoring= 'accuracy',    # 검증할때마다 정확도를 이용하여 모델을 평가
    cv = cv_folds,      # 교차 검증은 횟수는 KFold의 값들을 이용
    verbose = 1,        # 진행상황을 간단한 로그로 표시 
    refit = True,       # 베스트 파라미터를 이용하여 재학습 
    return_train_score= True,   # 학습 데이터의 성능을 확인
    n_jobs=-1           # 해당 코드를 이용하여 작업시 사용할 코어는 모두
)
grid_cls.fit(X_train, Y_train)

In [ ]:
print(grid_cls.score(X_test, Y_test))

In [ ]:
# print(grid_cls.cv_results_)
pd.DataFrame(grid_cls.cv_results_).sort_values(
    "mean_test_score", ascending=False
)

In [ ]:
# pipe + kfold + grid를 이용한 회귀 분석 
# csv에 있는 boston 데이터 로드 
boston = pd.read_csv("../csv/boston.csv")

In [ ]:
x = boston.drop('Price', axis=1).values
y = boston['Price']

In [ ]:
# train, test 나눌때 회귀 분석이기때문에 계층화 사용하지 않음
X_train, X_test, Y_train, Y_test = train_test_split(
    x, y, random_state= 42, test_size=0.2
)

In [ ]:
# 폴드화에서도 계층화가 필요 없기때문에 KFold 사용
cv_folds = KFold(n_splits=5, shuffle = True, random_state=42)

In [ ]:
pipe_reg = Pipeline(
    [
        ('stdscaler', StandardScaler()), 
        ('svm_reg', SVR())
    ]
)

In [ ]:
# SVR() 모델의 파라미터를 C는 [1, 10, 100]
# kenrle은 ['rbf', 'linear]
# epsilon은 [0.1, 0.2, 0.5]
# 파라미터 조합 
params_reg = {
    "svm_reg__C" : [1, 10, 100], 
    "svm_reg__kernel" : ['rbf', 'linear'], 
    "svm_reg__epsilon" : [0.1, 0.2, 0.5]
}

In [ ]:
# GridSearchCV를 이용하여 최적화 파라미터를 확인 
# scoring은 'neg_mean_squared_error'
grid_reg = GridSearchCV(
    estimator= pipe_reg, 
    param_grid= params_reg, 
    scoring='neg_mean_squared_error', 
    cv = cv_folds, 
    verbose= 1, 
    refit = True, 
    return_train_score=True
)
grid_reg.fit(X_train, Y_train)